In [1]:
from selenium.webdriver import Chrome
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

In [2]:
driver = Chrome()
driver.get('https://twitter.com/i/flow/login')

In [3]:
username = driver.find_element(By.XPATH,'//input[@name = "text"]')
username.send_keys('nad.yul2002@gmail.com')

In [4]:
username.send_keys(Keys.RETURN)

In [5]:
my_password = getpass()

········


In [6]:
password = driver.find_element(By.XPATH, '//input[@name = "password"]')
password.send_keys(my_password)

In [7]:
password.send_keys(Keys.RETURN)

In [8]:
search_input = driver.find_element(By.XPATH, '//input[@aria-label="Search query"]')
search_input.send_keys('Washington Post')
search_input.send_keys(Keys.RETURN)

In [14]:
driver.find_element_by_link_text('People').click()
nytimes_input = driver.find_element(By.XPATH, '//span[text() = "The Washington Post"]')
nytimes_input.click()


<ipython-input-14-a868ae6325a0>:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_link_text('People').click()


In [15]:
cards = driver.find_elements(By.XPATH, './/article[@data-testid="tweet"]')
card = cards[0]

In [16]:
#Tweet Publisher
card.find_element(By.XPATH, './/span').text

'Pinned Tweet'

In [17]:
#Tweet Date
card.find_element(By.XPATH, './/time').get_attribute('datetime')

'2021-12-07T22:16:24.000Z'

In [18]:
#Tweet Content 
comment = card.find_element(By.XPATH, './/div[1]/div[1]/div[1]/div[2]/div[2]/div[2]').text
responding = card.find_element(By.XPATH, './/div[1]/div[1]/div[2]').text
comment+responding

'In the middle of the 19th century, hundreds of thousands of new Americans flooded into New York. \n\nThey found homes in buildings like this one, on Orchard Street on Manhattan’s Lower East Side. https://wapo.st/3GhSbOA\nGIF\n667\n445\n1.1KPinned Tweet'

In [19]:
#Tweet Reply Count
card.find_element(By.XPATH, './/div[@data-testid="reply"]').text

'667'

In [20]:
#Tweet Retweet
card.find_element(By.XPATH, './/div[@data-testid="retweet"]').text

'445'

In [21]:
#Tweet Likes
card.find_element(By.XPATH, './/div[@data-testid="like"]').text

'1.1K'

In [22]:
def get_tweet_data(card):
    """Extract data from tweet"""
    username = card.find_element(By.XPATH, './/span').text
    handle = card.find_element(By.XPATH, './/span[contains(text(), "@")]').text
    try:
        postdate = card.find_element(By.XPATH, './/time').get_attribute('datetime')
    except NoSuchElementException:
        return
    comment = card.find_element(By.XPATH, './/div[1]/div[1]/div[1]/div[2]/div[2]/div[2]').text
    responding = card.find_element(By.XPATH, './/div[1]/div[1]/div[2]').text
    text = comment+responding
    reply_cnt = card.find_element(By.XPATH, './/div[@data-testid="reply"]').text
    retweet_cnt = card.find_element(By.XPATH, './/div[@data-testid="retweet"]').text
    likes_cnt = card.find_element(By.XPATH, './/div[@data-testid="like"]').text
    
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, likes_cnt)
    return tweet

In [23]:
get_tweet_data(card)

('Pinned Tweet',
 '@washingtonpost',
 '2021-12-07T22:16:24.000Z',
 'In the middle of the 19th century, hundreds of thousands of new Americans flooded into New York. \n\nThey found homes in buildings like this one, on Orchard Street on Manhattan’s Lower East Side. https://wapo.st/3GhSbOA\nGIF\n667\n445\n1.1KPinned Tweet',
 '667',
 '445',
 '1.1K')

In [24]:
tweet_data = []

for card in cards:
    data = get_tweet_data(card)
    if data:
        tweet_data.append(data)

tweet_data

[('Pinned Tweet',
  '@washingtonpost',
  '2021-12-07T22:16:24.000Z',
  'In the middle of the 19th century, hundreds of thousands of new Americans flooded into New York. \n\nThey found homes in buildings like this one, on Orchard Street on Manhattan’s Lower East Side. https://wapo.st/3GhSbOA\nGIF\n667\n445\n1.1KPinned Tweet',
  '667',
  '445',
  '1.1K'),
 ('The Washington Post',
  '@washingtonpost',
  '2021-12-27T18:12:33.000Z',
  'Somalia’s election standoff intensifies, raising risk of political violence\nwashingtonpost.com\nSomalia’s election standoff intensifies, raising risk of political violence\nMohamed Abdullahi "Farmajo" Mohamed, Somalia\'s president, attempted to sideline his main rival, Prime Minister Mohamed Hussein Roble, by suspending his powers.\n10\n13\n17The Washington Post\n@washingtonpost\n·\n9m\nSomalia’s election standoff intensifies, raising risk of political violence\nwashingtonpost.com\nSomalia’s election standoff intensifies, raising risk of political violence\n

In [25]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

In [29]:
#get all tweets on the page

data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements(By.XPATH, './/article[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
    
    scroll_attempt = 0
    while True:            
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(1)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1

            if scroll_attempt >=3:
                scrolling = False
                break
            else:
                sleep(2)
        else:
            last_position = curr_position
            break

In [30]:
data

[('Pinned Tweet',
  '@washingtonpost',
  '2021-12-07T22:16:24.000Z',
  'In the middle of the 19th century, hundreds of thousands of new Americans flooded into New York. \n\nThey found homes in buildings like this one, on Orchard Street on Manhattan’s Lower East Side. https://wapo.st/3GhSbOA\nGIF\n667\n445\n1.1KPinned Tweet',
  '667',
  '445',
  '1.1K'),
 ('The Washington Post',
  '@washingtonpost',
  '2021-12-27T18:12:33.000Z',
  'Somalia’s election standoff intensifies, raising risk of political violence\nwashingtonpost.com\nSomalia’s election standoff intensifies, raising risk of political violence\nMohamed Abdullahi "Farmajo" Mohamed, Somalia\'s president, attempted to sideline his main rival, Prime Minister Mohamed Hussein Roble, by suspending his powers.\n11\n14\n22The Washington Post\n@washingtonpost\n·\n11m\nSomalia’s election standoff intensifies, raising risk of political violence\nwashingtonpost.com\nSomalia’s election standoff intensifies, raising risk of political violence\

In [32]:
with open('WashingtonPost_tweets.csv', 'w', newline = '', encoding = 'utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Comments', 'Likes', 'Retweets', 'Text']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)